# **Entrenamiento del Modelo GNN**

Pipeline de entrenamiento para el modelo GNN de ordenamiento de nodos.

In [ ]:
%run librerias.ipynb

In [ ]:
%run gnn_model.ipynb

## **1. FUNCIÓN DE ENTRENAMIENTO**

In [ ]:
def train_gnn(model, data, optimizer, epochs=100):
    model.train()
    best_loss = float("inf")

    for epoch in range(epochs):
        optimizer.zero_grad()

        scores = model(data.x, data.edge_index)
        ordering = torch.argsort(scores, descending=True).tolist()

        num_colors = greedy_coloring_gnn(
            data.edge_index, data.num_nodes, ordering
        )

        loss = torch.tensor(num_colors, dtype=torch.float32, requires_grad=True)
        loss.backward()
        optimizer.step()

        if loss.item() < best_loss:
            best_loss = loss.item()

        if epoch % 10 == 0:
            print(f"Epoch {epoch:03d} | Colors: {num_colors}")

    return best_loss

## **2. PREPARACIÓN DE DATOS**

In [ ]:
data = preparar_datos_gnn(num_nodes=1000, num_edges=300)
print(f"Datos preparados: {data.num_nodes} nodos, {data.edge_index.shape[1]} aristas")

## **3. CONFIGURACIÓN DEL OPTIMIZADOR**

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("Optimizador configurado: Adam (lr=0.01)")

## **4. ENTRENAMIENTO**

In [ ]:
print("Iniciando entrenamiento...")
print("="*60)
best_loss = train_gnn(model, data, optimizer, epochs=50)
print("="*60)
print(f"Entrenamiento completado. Mejor loss: {best_loss}")